# NLP con Deep Learning: Transformers

- [Curso simple Coursera](https://www.coursera.org/)

- [Curso Avanzado Stanford](https://web.stanford.edu/class/cs224n/index.html#schedule)

- [Implementar Transformers para Clasificacion](https://huggingface.co/transformers/v3.2.0/custom_datasets.html)

In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
#device = "mps" # para mac M1 en adelante
device = "cuda:0" if torch.cuda.is_available() else "cpu" #para gpu
print(device)

cuda:0


In [ ]:
df_train = pd.read_parquet('https://github.com/amiune/amiune.github.io/raw/master/movie-genre-prediction/train.parquet', engine='pyarrow')
df_train.head()

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action


In [ ]:
df_train.shape

(54000, 4)

In [ ]:
df_train.genre = df_train.genre.astype('category')

id2label = dict(zip(df_train.genre.cat.codes, df_train.genre))
print(id2label)
label2id = dict(zip(df_train.genre, df_train.genre.cat.codes))
print(label2id)

{4: 'fantasy', 5: 'horror', 3: 'family', 8: 'scifi', 0: 'action', 2: 'crime', 1: 'adventure', 6: 'mystery', 7: 'romance', 9: 'thriller'}
{'fantasy': 4, 'horror': 5, 'family': 3, 'scifi': 8, 'action': 0, 'crime': 2, 'adventure': 1, 'mystery': 6, 'romance': 7, 'thriller': 9}


In [ ]:
num_classes = len(df_train.genre.value_counts())
print(num_classes)

10


In [ ]:
#df_train["text"] = df_train["movie_name"].str.lower() + ". " + df_train["synopsis"].str.lower()
df_train["text"] = df_train["movie_name"] + ". " + df_train["synopsis"]

In [ ]:
df_train.iloc[0,:]["text"]

'Super Me. A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.'

In [ ]:
def read_columns(df, text_column, label_column):
    texts = df[text_column].tolist()
    labels = df[label_column].cat.codes.tolist()
    return texts, labels

In [ ]:
train_texts, train_labels = read_columns(df_train.iloc[0:,:], "synopsis","genre")
print(len(train_texts),len(train_labels))

54000 54000


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.05, stratify=train_labels, random_state=42)
print(len(train_texts),len(train_labels))
print(len(val_texts),len(val_labels))

51300 51300
2700 2700


# HF Transformers

In [ ]:
!pip install transformers[torch] --quiet
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
print(type(train_encodings))
print(train_encodings.keys())

<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask'])


In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item es un diccionario que contiene tres keys: 'input_ids', 'attention_mask' y 'labels'
        #cada key contiene el tensor correspodiente al indice idx
        item = {}
        item['input_ids'] = torch.tensor(self.encodings['input_ids'][idx]).to(device)
        item['attention_mask'] = torch.tensor(self.encodings['attention_mask'][idx]).to(device)
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)
print(len(train_dataset),len(val_dataset))
print(train_dataset[0])

51300 2700
{'input_ids': tensor([     0,   5987,    599,     62,    561,  37356,     23,   2315,   4440,
         31738,    644,     68,  99388,   5094,   5781,   5386,      4,  13286,
         56890,    165,  10810,  16099,  88523,  21122,  81110,      4,   1421,
            68,   1697,   7372, 227144,      4,     68,    198,    493,  12365,
          9703,      4,    749,     23,     68,  88007,     33, 102386,  10110,
           510,      5,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1], device='cuda:0'), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
!pip install evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00


In [ ]:
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

#https://huggingface.co/transformers/v4.2.2/main_classes/trainer.html#trainingarguments
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    learning_rate=0.000004,
    weight_decay=0.6,
    dataloader_pin_memory=False,     # True if possible for faster training
    save_steps=1_000_000_000,         # dont save checkpoints
    save_total_limit=0,
    #load_best_model_at_end=True,
    #metric_for_best_model="eval_accuracy"
    evaluation_strategy = "epoch",
    output_dir="./results"
)

model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=num_classes).to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.ou

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.715400,1.664028,0.408519
2,1.567200,1.647428,0.422222
3,1.508600,1.659789,0.420741


TrainOutput(global_step=19239, training_loss=1.632925127454629, metrics={'train_runtime': 13647.9083, 'train_samples_per_second': 11.276, 'train_steps_per_second': 1.41, 'total_flos': 2.82934364755716e+16, 'train_loss': 1.632925127454629, 'epoch': 3.0})

In [ ]:
df_test = pd.read_parquet('https://github.com/amiune/amiune.github.io/raw/master/movie-genre-prediction/test.parquet', engine='pyarrow')
df_test.head()

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action


In [ ]:
df_test["text"] = df_test["movie_name"] + ". " + df_test["synopsis"]
test_texts = df_test["text"].to_list()
test_texts[0]

"A Death Sentence. 12 y.o. Ida's dad'll die without a DKK1,500,000 operation. Ida plans to steal the money from the bank, her mom installed alarm systems in. She'll need her climbing skills, her 2 friends and 3 go-karts."

In [ ]:
test_preds = []
test_probs = np.zeros((len(test_texts),num_classes))
for i in range(len(test_texts)):
  val_encoding = tokenizer(test_texts[i], truncation=True, padding=True, return_tensors="pt").to(device)
  outputs = model(**val_encoding)
  logits = outputs.logits.cpu().detach().numpy()
  # obtener prediccion
  test_preds.append(id2label[np.argmax(logits)])
  # calcular probabilidades
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(outputs.logits.squeeze().cpu())
  for j in range(num_classes):
    test_probs[i,j] = probs[j]

from google.colab import files

tmp_dict = {"id":df_test["id"]}
for j in range(num_classes):
    tmp_dict[id2label[j]] = test_probs[:,j]
df_probs = pd.DataFrame(tmp_dict)
df_probs.to_csv("roberta_large_probs.csv", index=False)
files.download('roberta_large_probs.csv')

df_submission = pd.DataFrame({"id":df_test["id"],"genre":test_preds})
df_submission.to_csv("submission7.csv", index=False)
files.download('submission7.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>